# Failed time discretization and interpolation

# Test for time-space recurring scheme with conditional equation depending on spatial position

# Implementation of former scheme 

## NO nonlinearity > NO hrad/hinf actualization

## hrad and hinf calculated for all time-step prior to time-space profile calculation

$$h_{rad}=\Psi_{sky}\varepsilon\sigma\left(T_{surface}^2+T_{sky}^2\right)\left(T_{surface}+T_{sky}\right)$$  

With:  
-- $h_{rad}$ the abbreviated parameter for outgoing radiative coefficient []  
-- $\Psi_{sky}$ the sky view factor []  
-- $\varepsilon$ the infrared emissivity of the surface []  
-- $\sigma=5.67e^{-8}$ the Stefan-Boltzmann constant [W/m²/K⁴]  
-- $T_{surface}$ the pavement surface temperature [K]

**Air film properties** inputs are for interpolation and/or extrapolation of quantities at the air film temperature [K]:

$$T_{film}=\frac{T_{surface}+T_{atm}}{2}$$

## Convection heat transfer calculation

### 1st: we check if we have laminar or turbulent air flow  

$$Re=\frac{U_{film}L}{\nu_{film}}$$

If $Re<5e^5$, then the flow is laminar.  
Else it is turbulent.  

### 2nd: we define the fluid Nusselt number accordingly  

For laminar flow:  
$$Nu_{laminar}=0.664\left[Pr^{1/3}Re^{0.5}\right]$$

For turbulent flow:
$$Nu_{turbulent}=0.037\left[Pr^{1/3}Re^{0.8}\right]$$

### 3rd: we calculate the convective heat transfer coefficient of air  

For laminar flow:  
$$h_{laminar}=0.664\left[k_{film}Pr^{1/3}\nu_{film}^{-0.5}L^{-0.5}U_{film}^{0.5}\right]$$
$$h_{laminar}=Nu_{laminar}\frac{k_{film}}{L}$$

For turbulent flow:
$$h_{turbulent}=0.037\left[k_{film}Pr^{1/3}\nu_{film}^{-0.8}L^{-0.2}U_{film}^{0.8}\right]$$
$$h_{turbulent}=Nu_{turbulent}\frac{k_{film}}{L}$$

With:  
-- $Re$ the Reynolds number of air []  
-- $U_{film}$ the wind velocity [m/s]  
-- $L$ the characteristic length of the pavement [m]  
-- $\nu_{film}$ the kinematic viscosity of air [m²/s]  
-- $Nu_{xxx}$ the Nusselt number of air []  
-- $Pr$ the Prandtl number of air []  
-- $h_{xxx}$ the convective heat transfer coefficient of air []  
-- $k_{film}$ the thermal conductivity of air [W/m/K]  

## Stability verification

Calculate Courant-Friedrichs-Lewy - CFL criterion at all time

$$\Delta t \leq \frac{\rho_{surf}c_{surf}\Delta x^2}{2\left(h_{rad}\Delta x + h_{laminar}\Delta x + k_{surf}\right)}$$

OR  

$$\Delta t \leq \frac{\rho_{surf}c_{surf}\Delta x^2}{2\left(h_{rad}\Delta x + h_{turbulent}\Delta x + k_{surf}\right)}$$

With:  
-- $\Delta t$ the temporal discretization step [s]  
-- $\rho_{surf}$ the density of the surface layer [kg/m³]  
-- $c_{surf}$ the specific heat capacity of the surface layer [J/kg/K]  
-- $\Delta x$ the spatial discretization step [m]  
-- $k_{surf}$ the thermal conductivity of the surface layer [W/m/K]

Check if CFL criterion is respected (ie. CFL > time interval chosen) at all time

If all elements in array "stability" are True, then np.all() == True

# Time-Space matrix calculation

In [ ]:
for time in range(0,profile.shape[0]):
    for depth in range(profile.shape[1]):
        depth_m=round(depth*node_spacing,2)
        if depth_m==0:
            profile[time,depth]=(2*time_step)/(density[0]*spec_heat_capacity[0]*node_spacing) * \
                                (solar_view_factor*(1-surface_albedo)*solar_rad_interp[time] + \
                                 convective_heat_air[time]*(temp_atm_interp[time]-profile[time-1,depth]) + \
                                 hrad[time]*(temp_sky_interp[time]-profile[time-1,depth]) + \
                                 (thermal_conductivity[0]/time_step)*(profile[time-1,depth+1]-profile[time-1,depth])) + \
                                profile[time-1,depth]
            
        elif depth_m<=pavement_depth[-1]:
            for interface in range(len(pavement_depth)):
                if depth_m<pavement_depth[interface]:
                    profile[time,depth] = (time_step)/(density[interface]*spec_heat_capacity[interface]*node_spacing) * \
                                          (thermal_conductivity[interface]/node_spacing * \
                                          (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                          profile[time-1,depth]
                    break
                    
                elif depth_m==pavement_depth[interface]:
                    profile[time,depth]=0.5*((2*node_spacing*thermal_conductivity[interface]+ \
                                              thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                              profile[time,depth-1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                            +(2*node_spacing*thermal_conductivity[interface+1]+ \
                                              thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                              profile[time,depth+1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                            )
                    if time==1: print('interface nº'+str(interface))
                    break
                    
        elif depth_m!=ground_layer_depth:
            profile[time,depth] = (time_step)/(density[-1]*spec_heat_capacity[-1]*node_spacing) * \
                                          (thermal_conductivity[-1]/node_spacing * \
                                          (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                          profile[time-1,depth]
            
        else:
            profile[time,depth] = deep_ground_temp
            if time==1: print('deep ground temp reached')
